In [1]:
import pandas
raw = pandas.read_excel('FSADataCenter_Demographics_2015-16Cycle.xls')

In [2]:
raw

,Free Application for Federal Student Aid (FAFSA) By Various Demographic Characteristics 2015/16 Application Cycle,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Data Element and Value,Initial Application,NaN,NaN,NaN,NaN,NaN,NaN,Final Transaction
1,NaN,2015,NaN,NaN,NaN,2016,NaN,Total,Total
2,NaN,Q1,Q2,Q3,Q4,Q5,Q6+,NaN,NaN
3,Total (Applicant Count),8681436,5001114,3456193,1360716,832592,425713,19757764,19757764
4,Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Female,5280276,3050260,2029742,820497,494181,261826,11936782,11963730
6,Male,3378128,1923019,1395750,526270,327404,156278,7706849,7708027
7,Blank,23032,27835,30701,13949,11007,7609,114133,86007
8,Age,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,18 or Less,1238394,294440,170016,121047,79052,55099,1958049,1958328
